In [ ]:
import logging
import sys

import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.display as afwDisplay
import lsst.daf.butler as dafButler

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=False, correctGain=False)
    return oim

In [ ]:
dataPath = "/repo/embargo"
instrument = "LSSTCam"
butler = dafButler.Butler(dataPath, 
                          collections=["LSSTCam/raw/all", "LSSTCam/calib/DM-49832/unbounded", "LSSTCam/runs/nightlyValidation"],
                          instrument=instrument)

In [ ]:
day_obs = 20250415
seq_num = 60
dataId = {"instrument": instrument,
          "exposure.day_obs": day_obs, "exposure.seq_num": seq_num}
# dataType = 'raw'
# dataType = 'post_isr_image'
dataType = 'preliminary_visit_image'
if dataType == 'raw':
    callback = myCallback
else:
    callback = None

In [ ]:
camera = butler.get('camera', instrument=instrument)

In [ ]:
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('asinh', 'zscale')

mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument,
                                                       day_obs=day_obs, seq_num=seq_num,
                                                       verbose=True, callback=callback,
                                                       background=np.nan),
                              binSize=16, display=disp, overlay=False,
                              title="%d %d %s" % (day_obs, seq_num, dataType))

In [ ]:
# fig.savefig("pvi_img.png")